# Debugging Notbook
If the debugger is attached, we can use the debugger to touch vscode debug points, when calling routes.

In [ ]:
import httpx
import environ
import importlib
from pathlib import Path
from rich import print

In [ ]:
# ! Set the environment mode you want to use here
# MODE = "production"
MODE = "develop"

In [ ]:
# parent directory cwd for incomplicated module level imports
while not (Path.cwd() / "manage.py").exists():
    parent = Path.cwd().parent
    %cd $parent

from core.setup import setup_django_settings

# load environment variables depending on mode
setup_django_settings(mode=MODE)
from django.conf import settings
HOST = settings.PROJECT_FQDN

In [ ]:
# load urls routes..
from io import StringIO

output = StringIO()
from django.core.management import call_command

_ = call_command("show_urls", stdout=output)  # from django_extensions.management.commands import show_urls

urls = []
for string in output.getvalue().split("\n"):
    route, *_ = string.split("\t")
    if not route.startswith("/_") and "<path>" not in route:
        urls.append(f"{HOST}{route}")
urls = list(set(urls))

In [ ]:
# create a user
email = "user@example.com"
password = "secret"
delete_all_existing = False

from django.contrib.auth import get_user_model
from django.db import IntegrityError

UserModel = get_user_model()
from asgiref.sync import sync_to_async


def create_user(email, password, delete_all_existing: bool = False):
    if delete_all_existing:
        UserModel.objects.all().delete()
    try:
        user = UserModel.objects.create(username=email, email=email)
    except IntegrityError:
        user = UserModel.objects.get(email=email)
    user.set_password(password)
    user.save()
    return user


user = await sync_to_async(create_user)(email, password, delete_all_existing=delete_all_existing)


In [5]:
# all the urls available for django
print(urls)

[
    'http://localhost:8000/api/protected/',
    'http://localhost:8000',
    'http://localhost:8000/api/auth/token/',
    'http://localhost:8000/api/'
]

In [ ]:
# if the authentication app is installed, we can login
url = [url for url in urls if "auth/token" in url][0]  # login

r = httpx.post(url, json={"email": email, "password": password})
print(f"{url} - {r.status_code}")

In [ ]:
print(r.json())
print(r.cookies)